In [1]:
import pandas as pd
import numpy as np
import regex as re
import os

In [2]:
#peeking into data
pd.read_html('monthly_sales_by_category/2024.html')[2]

,Type,Group,Maker/Brand,Jan.,Feb.,Mar.,Apr.,May,Jun.,First Half (Jan.-Jun.),Jul.,Aug.,Sep.,Oct.,Nov.,Dec.,Second Half (Jul.-Dec.),Adjusted value,Total
0,Cars,VW Group,VW,1879,1631,1847,1183,1610,1656,9806.0,1766.0,1876.0,-,-,-,-,3642.0,-,13448.0
1,Cars,VW Group,Audi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,-,-,-,NaN,-,NaN
2,Cars,VW Group,Skoda,1242,1028,1374,1266,1542,1231,7683.0,793.0,1125.0,-,-,-,-,1918.0,-,9601.0
3,Cars,VW Group,Porsche (2013-),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,-,-,-,NaN,-,NaN
4,Cars,VW Group,VW Group Total,3121,2659,3221,2449,3152,2887,17489.0,2559.0,3001.0,-,-,-,-,5560.0,-,23049.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,Buses,Small and Medium OEM,EKA Mobility,-,-,-,1,1,1,3.0,NaN,NaN,-,-,-,-,NaN,-,3.0
106,Buses,Small and Medium OEM,Olectra,45,43,42,0,0,0,130.0,NaN,NaN,-,-,-,-,NaN,-,130.0
107,Buses,Small and Medium OEM,Small and Medium OEM Total,45,43,42,1,1,1,133.0,NaN,NaN,-,-,-,-,NaN,-,133.0
108,Buses,Buses Total,Buses Total,6965,6558,6525,5494,5743,5617,36902.0,NaN,NaN,-,-,-,-,NaN,-,36902.0


## Pipeline to clean and store data

In [3]:
#creating a dictionary to hold all the years from the source html files
sale={}

#as monthly sales data is obtained for the years 2007 to 2024(till Aug.)
for year in range(2024,2006,-1):
    
    sale.update({year:pd.read_html('monthly_sales_by_category/'+str(year)+'.html')[2]})
    
    #dropping columns and rows that arent needed
    sale[year].drop(columns=['Group','First Half (Jan.-Jun.)','Second Half (Jul.-Dec.)',
                         'Adjusted value','Total'],axis=1,inplace=True)
    
    sale[year].drop(index=list(range(sale[year][sale[year]['Type']=='Medium/Heavy Trucks'].index[0],len(sale[year]))),
                axis=0,inplace=True)
    
    #to replace complex values with the category value
    for indx,value in zip(range(0,len(sale[year])),sale[year]['Type'].values):
        if value == 'Cars':
            sale[year].iloc[indx,0] = 'PassengerCar'
            continue
        elif value == 'UVs/MPVs':
            sale[year].iloc[indx,0] = 'SUV'
            continue
        elif value == 'Light Trucks':
            sale[year].iloc[indx,0] = 'LightTruck'
            continue
    
    #to drop rows which doesnt contain maker/brand name
    sale[year].drop(index=sale[year][sale[year]['Maker/Brand'].isna()].index,axis=0,inplace=True)
    
    #to drop all subtotals and keep data constituting only to maker/brand
    sale[year].drop(index=sale[year][['Maker/Brand']][sale[year]['Maker/Brand'].str.contains('Total')].index,
                axis=0,inplace=True)
    
    #replacing '-' to NaN, so that it can be replaced with mean value
    sale[year].iloc[:,2:-4] = sale[year].iloc[:,2:-4].replace('-',np.nan)
    
    #dropping rows that has NaN for more than 2 months, and retain data containing solid info
    #as first 2 columns have already been taken care, 
    #so setting thresh=12, to have 12 out of the 14 values filled in the row
    sale[year].dropna(axis=0,thresh=8,inplace=True)
    
    #cleaning maker/brand name that has suffix containing years operated 
    for value in sale[year][sale[year]['Maker/Brand'].str.contains(r'\W\d{4}-\W')][['Maker/Brand']].iterrows():
        to_fill= value[1].values[0][:-8]
        sale[year].loc[value[0],'Maker/Brand']=to_fill
    for value in sale[year][sale[year]['Maker/Brand'].str.contains(r'\W-\d{4}\W')][['Maker/Brand']].iterrows():
        to_fill= value[1].values[0][:-8]
        sale[year].loc[value[0],'Maker/Brand']=to_fill
    
    #as this work is carried out in sept 2024, 
    #so considering data upto Aug only for year 2024, and all months for rest of the years
    #also replacing unfilled data '-' with 0 for 2024
    if year==2024:
        months = sale[year].columns[2:-4]
        sale[year].replace('-',0,inplace=True)
    else:
        months = sale[year].columns[2:]
        
    #converting datatype of the months columns
    sale[year][months] = sale[year][months].replace('-',np.nan)
    sale[year][months] = sale[year][months].astype('float')
    
    #replacing missing values in the months with row mean value
    sale[year][months] = sale[year][months].apply(lambda row : row.fillna(np.ceil(row.mean())),axis=1)
    
    #crating column to contain year, annual total & Avg sales per maker/brand
    sale[year].insert(2,'Year',year)
    sale[year]['Annual']=sale[year][months].sum(axis=1)
    sale[year]['Monthly Avg']=np.round(sale[year][months].mean(axis=1))
    
    #after cleaning all the data, resetting index values
    sale[year].reset_index(drop=True,inplace=True)
    
    #storing the sheet into a csv file
    file_name = os.getcwd()+'/cleaned/'+str(year)+'_monthly_category_sales.csv'
    sale[year].to_csv(file_name)
    
    #creating master data table containing data of all the available years
    if year==2024:
        master_data = sale[year]
    else:
        master_data = pd.concat([master_data,sale[year]],axis=0,ignore_index=True)
    

#storing master data to csv
master_data.to_csv(os.getcwd()+'/cleaned/'+'monthly_sales_master_data.csv')

In [4]:
pd.read_csv(os.getcwd()+'/cleaned/2010_monthly_category_sales.csv',index_col=0)

,Type,Maker/Brand,Year,Jan.,Feb.,Mar.,Apr.,May,Jun.,Jul.,Aug.,Sep.,Oct.,Nov.,Dec.,Annual,Monthly Avg
0,PassengerCar,VW,2010,319.0,377.0,197.0,1489.0,1796.0,2032.0,2597.0,3531.0,4663.0,4343.0,4612.0,5772.0,31728.0,2644.0
1,PassengerCar,Audi,2010,190.0,212.0,153.0,265.0,183.0,247.0,285.0,273.0,292.0,347.0,356.0,396.0,3199.0,267.0
2,PassengerCar,Skoda,2010,1881.0,1805.0,1824.0,1285.0,1381.0,1638.0,1222.0,1511.0,1467.0,1661.0,1841.0,2502.0,20018.0,1668.0
3,PassengerCar,Toyota,2010,947.0,877.0,1120.0,904.0,710.0,547.0,883.0,1068.0,1120.0,987.0,793.0,1351.0,11307.0,942.0
4,PassengerCar,Nissan,2010,30.0,23.0,21.0,24.0,14.0,35.0,955.0,1205.0,1188.0,1087.0,1048.0,1073.0,6703.0,559.0
5,PassengerCar,Chevrolet,2010,7819.0,9449.0,9464.0,8904.0,6712.0,7932.0,5298.0,6278.0,6847.0,8250.0,6788.0,6611.0,90352.0,7529.0
6,PassengerCar,Hyundai,2010,29601.0,31000.0,31501.0,28501.0,27151.0,27366.0,28811.0,28601.0,31751.0,34651.0,31501.0,26066.0,356501.0,29708.0
7,PassengerCar,Ford,2010,2138.0,2875.0,9063.0,7226.0,7820.0,7059.0,8473.0,7701.0,8098.0,8733.0,7305.0,4194.0,80685.0,6724.0
8,PassengerCar,Honda,2010,5907.0,6146.0,5862.0,3507.0,4032.0,4547.0,4642.0,5496.0,7640.0,5246.0,4070.0,5109.0,62204.0,5184.0
9,PassengerCar,Fiat,2010,2302.0,2256.0,2161.0,1800.0,2163.0,2137.0,2301.0,1812.0,1650.0,2070.0,1023.0,275.0,21950.0,1829.0


___

In [5]:
#peeking into data
pd.read_html('annual_sales_by_model/fiat_jeep_citroen.html')[1]

,Group,Maker/Brand,Segment,Model,PowerTrain,Country,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Stellantis,Jeep (2021-),SUV-C,Compass (Jeep (2009-)),NaN,India,-,-,-,-,-,-,-,-,-,11652,9866,4054
1,Stellantis,Jeep (2021-),SUV-C,Compass (Jeep (2009-)),Compass (Jeep (2009-)) Total,Compass (Jeep (2009-)) Total,-,-,-,-,-,-,-,-,-,11652,9866,4054
2,Stellantis,Jeep (2021-),SUV-C,SUV-C Total,SUV-C Total,SUV-C Total,-,-,-,-,-,-,-,-,-,11652,9866,4054
3,Stellantis,Jeep (2021-),SUV-D,Meridian,NaN,India,-,-,-,-,-,-,-,-,-,-,3451,2571
4,Stellantis,Jeep (2021-),SUV-D,Meridian,Meridian Total,Meridian Total,-,-,-,-,-,-,-,-,-,-,3451,2571
5,Stellantis,Jeep (2021-),SUV-D,SUV-D Total,SUV-D Total,SUV-D Total,-,-,-,-,-,-,-,-,-,-,3451,2571
6,Stellantis,Jeep (2021-),Jeep (2021-) Total,Jeep (2021-) Total,Jeep (2021-) Total,Jeep (2021-) Total,-,-,-,-,-,-,-,-,-,11652,13317,6625
7,Stellantis,Citroen (2021-),B,C3,EV,India,-,-,-,-,-,-,-,-,-,-,-,2360
8,Stellantis,Citroen (2021-),B,C3,NaN,India,-,-,-,-,-,-,-,-,-,-,5686,5738
9,Stellantis,Citroen (2021-),B,C3,C3 Total,C3 Total,-,-,-,-,-,-,-,-,-,-,5686,8098


## Pipeline to clean and store data

In [6]:
#creating a dictionary to hold data of all the models by brand
brand = {}

#just a parameter to differentiate first loop vs rest of the loops
flag=True

#parsing through all the files with nested for loop
for folders,subs,files in os.walk(os.getcwd()+'/annual_sales_by_model'):
    for file in files:
        
        #getting name of the file, which is also the brand name
        name = file.split('.')[0]
        
        #reading data into dictionary
        brand.update({name:pd.read_html(os.getcwd()+'/annual_sales_by_model/'+file)[1]})
        
        #dropping unwanted columns
        brand[name].drop(columns=['Group','Country'],inplace=True)
        
        #as the source data has mention only of Electric Vehicle (EV) / Hybrid Vehicle (HV),
        #the remaining null values are to be filled up with Internal Combustion Engine (ICE)
        brand[name]['PowerTrain'].fillna('ICE',inplace=True)
        
        #replacing similar category name with standard naming convention, 
        #to avoid same category in different naming convention
        brand[name]['PowerTrain'].replace('HV/MHV','HV',inplace=True)
        
        #dropping rows containing subtotal for each category
        brand[name].drop(index=brand[name][brand[name]['PowerTrain'].str.contains('Total')].index,inplace=True)
        
        #removing naming suffix from the values in Brand column
        for value in brand[name][brand[name]['Maker/Brand'].str.contains(r'\W\d{0,}-\d{0,4}\W')][['Maker/Brand']].iterrows():
            brand[name].loc[value[0],'Maker/Brand'] = value[1].values[0].split()[0]
        
        #dropping of rows which doesnt contain the model name
        brand[name].drop(index=brand[name][brand[name]['Model'].isna()].index,axis=0,inplace=True)
        
        #removing naming suffix from the values in Model column
        for value in brand[name][brand[name]['Model'].str.contains(r'\W\D{0,}\W\d{0,}-\d{0,4}\W\W')][['Model']].iterrows():
            brand[name].loc[value[0],'Model'] = value[1].values[0].split()[0]
        
        #just taking year column names for future refrence
        years = brand[name].columns[4:]
        
        #replacing '-' with NaN to perform .dropna()
        brand[name][years] = brand[name][years].replace('-',np.nan)
        
        #dropping rows that doesnt contain any sales data
        brand[name].dropna(thresh=5,inplace=True)
        
        #replacing NaN to 0 for converting columns to float datatype
        brand[name].replace(np.nan,0,inplace=True)
        brand[name][years] = brand[name][years].astype('float')
        
        #in some cases, when parent company has got changed, 
        #the data has duplicate entry for the same model but with sales data split in between them
        #in order to merge those entries, we perform the below
        
        #check whether the data contains dulplicate entry of the model
        if len(brand[name][brand[name][brand[name].columns[:4]].duplicated()])>0:
            for items in brand[name].iterrows():
                #identifying the row details for the first row with duplicate entry
                if(list(items[1].values[:3])==list(brand[name][brand[name][brand[name].columns[:4]].duplicated()].values[0][:3])):
                    #adding the sales data part to the first row, so that all the required data is gathered in 1 entry
                    brand[name].loc[items[0],years]+=brand[name][brand[name][brand[name].columns[:4]].duplicated()][years].values[0]
                    break
            #dropping the second & further occurance of the entry
            brand[name].drop(index=brand[name][brand[name][brand[name].columns[:4]].duplicated()].index,inplace=True)
        
        #replacing similar categories under standard naming convention
        for indx,value in zip(range(0,len(brand[name])),brand[name]['Segment'].values):
            if re.search('SUV',value) or value=='MPV':
                brand[name].iloc[indx,1] = 'SUV'
                continue
            elif value in ['A','B','C','D','E','F']:
                brand[name].iloc[indx,1] = 'PassengerCar'
                continue
            elif value == 'Pickup Truck':
                brand[name].iloc[indx,1] = 'LightTruck'
                continue
        
        #dropping rows which arent categorized to Car sales data, (data pertaining to heavy trucks)
        brand[name].drop(index=brand[name][brand[name]['Segment']=='Unclassified'].index,axis=0,inplace=True)
        
        #adding up all the annual sales to Total column
        brand[name]['Total'] = brand[name][years].sum(axis=1)
        
        #resetting the index value, after cleaning all the data
        brand[name].reset_index(drop=True,inplace=True)
        
        #saving brand-model wise annual sales data into csv file
        file_name=os.getcwd()+'/cleaned/annual_sales_'+name+'.csv'
        brand[name].to_csv(file_name)
        
        #logic step for identifying first iteration
        if flag:
            #saving the brand-model wise data into master_data
            master_data = brand[name]
            flag = False
            continue
        else:
            master_data = pd.concat([master_data,brand[name]],ignore_index=True,axis=0)

#saving the master_data into csv format
master_data.to_csv(os.getcwd()+'/cleaned/annual_model_sales_master_data.csv')
master_data


,Maker/Brand,Segment,Model,PowerTrain,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,Total
0,Honda,PassengerCar,Brio,ICE,0.0,0.0,13246.0,10618.0,7260.0,5412.0,2277.0,3.0,0.0,0.0,0.0,0.0,38816.0
1,Honda,PassengerCar,Brio Amaze,ICE,0.0,0.0,65505.0,63831.0,35388.0,28314.0,67563.0,67715.0,37484.0,39697.0,46528.0,41292.0,493317.0
2,Honda,PassengerCar,Fit (Jazz),ICE,0.0,0.0,0.0,29644.0,34902.0,29890.0,18370.0,9512.0,3254.0,7259.0,4167.0,0.0,136998.0
3,Honda,PassengerCar,Civic,ICE,1099.0,74.0,0.0,0.0,0.0,0.0,0.0,4928.0,1225.0,6.0,0.0,0.0,7332.0
4,Honda,PassengerCar,Honda City,ICE,32404.0,21530.0,77346.0,82922.0,57619.0,62573.0,42045.0,28696.0,21826.0,34133.0,38335.0,20216.0,519645.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,Toyota,SUV,Vellfire,ICE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,333.0,478.0,683.0,565.0,2089.0
179,Toyota,LightTruck,Hilux,ICE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,904.0,904.0
180,Isuzu,SUV,MU-7,ICE,0.0,0.0,176.0,385.0,224.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,785.0
181,Isuzu,SUV,MU-X,ICE,0.0,0.0,0.0,0.0,0.0,799.0,952.0,577.0,173.0,65.0,52.0,67.0,2685.0


In [7]:

pd.read_csv(os.getcwd()+'/cleaned/annual_sales_toyota.csv',index_col=0)

,Maker/Brand,Segment,Model,PowerTrain,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,Total
0,Toyota,PassengerCar,Etios,ICE,42754.0,33586.0,26789.0,32511.0,31155.0,24061.0,21869.0,17236.0,0.0,0.0,0.0,0.0,229961.0
1,Toyota,PassengerCar,Etios Liva,ICE,31077.0,26396.0,21974.0,22139.0,15238.0,13552.0,15129.0,9083.0,0.0,0.0,0.0,0.0,154588.0
2,Toyota,PassengerCar,Vitz (Yaris),ICE,0.0,0.0,0.0,0.0,0.0,0.0,10799.0,2943.0,3807.0,2235.0,0.0,0.0,19784.0
3,Toyota,PassengerCar,Corolla,ICE,6912.0,4106.0,6748.0,7574.0,4860.0,4242.0,3268.0,1422.0,0.0,0.0,0.0,0.0,39132.0
4,Toyota,PassengerCar,Glanza,ICE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17946.0,20676.0,26051.0,33185.0,50537.0,148395.0
5,Toyota,PassengerCar,Prius,HV,11.0,1.0,3.0,1.0,4.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,24.0
6,Toyota,PassengerCar,Camry,HV,0.0,0.0,526.0,881.0,1254.0,809.0,334.0,672.0,413.0,500.0,1187.0,1819.0,8395.0
7,Toyota,PassengerCar,Camry,ICE,271.0,321.0,198.0,146.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1006.0
8,Toyota,SUV,Urban Cruiser,ICE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7600.0,27290.0,30214.0,0.0,65104.0
9,Toyota,SUV,Urban Cruiser Hyryder,HV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11864.0,42782.0,54646.0


___